In [1]:
#!pip install bitsandbytes
#!pip install git+https://github.com/huggingface/transformers.git
#!pip install git+https://github.com/huggingface/peft.git
#!pip install git+https://github.com/huggingface/accelerate.git
#!pip install datasets scipy ipywidgets matplotlib
#!pip install sentencepiece
#!pip install wandb

In [2]:
#!pip install trl
#!pip install flash-attn --no-build-isolation  # -> needs CUDA 11.6 or newer

In [1]:
def format_context(examples):
    examples['context_formatted'] = examples['context']['sentences']
    return examples

In [2]:
from datasets import load_dataset
from datasets import Dataset

dataset_path = "datasets/raft/" #"datasets/chatbot-qac-pairs/" "datasets/raft-dataset/"

#train_dataset = load_dataset('json', data_files=f'{dataset_path}dataset.json', split='train') #need to split as it will otherwise create a nested object
#eval_dataset = load_dataset('json', data_files=f'{dataset_path}dataset.json', split='validation')

#dataset = load_dataset('json', data_files=f'{dataset_path}dataset.json', split='train') # need to set split as it will otherwise create a nested object
 
dataset = Dataset.from_file(f"{dataset_path}data.arrow")

dataset = dataset.train_test_split(test_size=0.2).map(format_context)

train_dataset = dataset['train']
eval_dataset = dataset['test']

print(train_dataset)
print(eval_dataset)

Map:   0%|          | 0/361 [00:00<?, ? examples/s]

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction', 'context_formatted'],
    num_rows: 361
})
Dataset({
    features: ['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction', 'context_formatted'],
    num_rows: 91
})


In [3]:
import wandb, os
wandb.login()

wandb_project = "teamprojekt-chatbot-pruefungsamt"
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project
    #os.environ["WANDB_RESUME"] = "must"
    #os.environ["WANDB_ID"] = "20wlsltd"
    #os.environ["WANDB_ENTITY"] = "teamprojekt-chatbot-pruefungsamt"

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: benjaminbruenau (teamprojekt-chatbot-pruefungsamt). Use `wandb login --relogin` to force relogin


In [6]:
from huggingface_hub import notebook_login

# only necessary if model is not cached
#notebook_login()

'''
 pip install huggingface_hub["cli"]
huggingface-cli delete-cache
huggingface-cli login

'''

'\n pip install huggingface_hub["cli"]\nhuggingface-cli delete-cache\nhuggingface-cli login\n\n'

In [4]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-v0.3"#"../../../llms/mistral-7b-instruct-v0.2.Q4_K_M.gguf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    #padding_side="left",
    #add_eos_token=True,
    #add_bos_token=True,
)
#tokenizer.pad_token = tokenizer.eos_token

# Set reasonable default for models without max length
if tokenizer.model_max_length > 100_000:
  tokenizer.model_max_length = 2048
#tokenizer.model_max_length
tokenizer.pad_token = tokenizer.unk_token

## Format into necessary structure to avoid configuring and preprocessing by hand as much as possible

SFTTrainer expects the following instruction format:

```json
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
```

```python
def formatting_prompts_func(examples):
    texts = []
    for example in examples["messages"]:
        text = tokenizer.apply_chat_template(example, tokenize=false)
        texts.append(text)
    return { "text" : texts}
    
```


In [7]:
system_prompt = "You are a smart helpful assistant for the HTWG Konstanz. Answer the following question based only on the provided context. Always return the source of an information and it is mandatory to answer in GERMAN:\n\n"

def formatting_func(example):
    prompt = f"{system_prompt}Context: {example['context_formatted']}\n\nQuestion: {example['question']}"
    #prompt = f"[INST]{system_prompt} Context: {example['context']}\n\nQuestion: {example['question']} [/INST]"
    expected_answer = example["cot_answer"] # cot_answer / answer
    return {"prompt": prompt, "completion": expected_answer }

In [8]:
#train_dataset = train_dataset.map(formatting_func).remove_columns(['answer', 'question', 'context'])
#eval_dataset = eval_dataset.map(formatting_func).remove_columns(['answer', 'question', 'context'])

train_dataset = train_dataset.map(formatting_func).remove_columns(['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction', 'context_formatted'])
eval_dataset = eval_dataset.map(formatting_func).remove_columns(['id', 'type', 'question', 'context', 'oracle_context', 'cot_answer', 'instruction', 'context_formatted'])

print(train_dataset)
print(eval_dataset)

Map:   0%|          | 0/361 [00:00<?, ? examples/s]

Map:   0%|          | 0/91 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 361
})
Dataset({
    features: ['prompt', 'completion'],
    num_rows: 91
})


In [9]:
print(train_dataset['prompt'][0])

You are a smart helpful assistant for the HTWG Konstanz. Answer the following question based only on the provided context. Always return the source of an information and it is mandatory to answer in GERMAN:

Context: [['4 ermittelte \nTeilnote 3 abgezogen. Die Zahl der einzuladenden rangbesten Bewerber und Bewerberinnen \nbeträgt das Dr eifache der für die jeweilige Studienrichtung zur Verfügung stehenden \nStudienplätze. (4) Erstellung einer Rangliste für die Auswahlentscheidung nach § 10 \nFür die Auswahlentscheidung wird unter den Bewerbern und Bewerberinnen, die das \nAuswahlgespräch nach A bs. 2 Nr. 1 erfolgreich abgeschlossen haben, eine Rangliste je \nStudienrichtung nach einer Auswahlnote erstellt, in welche  \n1. die Vorauswahlnote gemäß Abs. 3 zu 50 vom Hundert sowie  \n2. die Teilnote 1 gemäß Abs. 2 Nr.', '1 Nr. 1 nachgewiesen sind \nDie Durchschnittsnote der Abschlussprüfung des grundständigen Hochschulstudiums nach \nAbs. 1 bildet die Auswahlnot e für die Auswahlentscheidu

In [10]:
print(train_dataset['completion'][1])

Step 1: Understand the question. The question is asking how the first sentence differs from the cases of § 3 Abs.

Step 2: Look at the context. The context states, ##begin_quote##"Abweichend von Satz 1 bildet in den Fällen des § 3 Abs."##end_quote## This translates to "Deviating from sentence 1 in the cases of § 3 Abs."

Step 3: Analyze the information. The context suggests that there is a deviation or difference in the cases of § 3 Abs from the first sentence.

Step 4: Formulate the answer. The answer should reflect the deviation or difference as indicated in the context.

<ANSWER>: The first sentence deviates or differs in the cases of § 3 Abs.


In [9]:
#tokenizer.apply_chat_template(train_dataset[0]['prompt'], tokenize=False)



messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

tokenizer.apply_chat_template(messages, tokenize=False)

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.


"<s>[INST] What is your favourite condiment? [/INST] Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen! </s><s>[INST] Do you have mayonnaise recipes? [/INST]"

## test model

In [11]:
def create_eval_prompt(query: str, context: str):
    system_prompt = "You are a smart helpful assistant for the HTWG Konstanz. Answer the following question based only on the provided context. It is mandatory to answer in GERMAN:\n\n"
    return f"[INST]{system_prompt}Context: {context}\n\nQuestion: {query}[/INST]"

eval_prompt = create_eval_prompt("""Who is the coordinator for the Data Science module at Hochschule Konstanz?""", "Modul-Koordination\nProf. Dr. O. D\u00fcrr")
eval_prompt

'[INST]You are a smart helpful assistant for the HTWG Konstanz. Answer the following question based only on the provided context. It is mandatory to answer in GERMAN:\n\nContext: Modul-Koordination\nProf. Dr. O. Dürr\n\nQuestion: Who is the coordinator for the Data Science module at Hochschule Konstanz?[/INST]'

In [12]:
# Init an eval tokenizer that doesn't add padding or eos token
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    add_bos_token=True,
)

model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(model.generate(**model_input, max_new_tokens=256, repetition_penalty=1.15)[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

# LoRA

In [11]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [12]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)
print(model)
model = get_peft_model(model, config)
print_trainable_parameters(model)
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )

# Training


In [13]:
'''
if torch.cuda.device_count() > 1:
    model.is_parallelizable = True
    model.model_parallel = True
'''
model = accelerator.prepare_model(model)

In [14]:
print(tokenizer.padding_side)
tokenizer

left


LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-v0.3', vocab_size=32768, model_max_length=2048, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[INST]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[/INST]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("[TOOL_CALLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	6: AddedToken("[AVAILABLE_TOOLS]", rstrip=F

In [15]:
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
from datetime import datetime

project = "chatbot-pruefungsamt-finetune"
run_type = "raft-completion-only"
run_name = base_model_id.split('/')[1] + f"-{run_type}-" + project
output_dir = "./finetune_runs/" + run_name



training_args = SFTConfig(
    output_dir=output_dir,
    report_to="wandb",
    warmup_steps=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    gradient_checkpointing=True,
    max_steps=500,
    learning_rate=2.5e-5, # Want a small lr for finetuning, about 10x smaller than the Mistral learning rate
    #gradient_accumulation_steps=2, # simulate larger batch sizes
    #per_device_train_batch_size=16,
    #learning_rate=2e-4,
    #lr_scheduler_type="cosine",
    #num_train_epochs=3,
    bf16=True,
    optim="paged_adamw_8bit",
    logging_steps=25,              # When to start reporting loss
    logging_dir="./logs",        # Directory for storing logs
    save_strategy="steps",       # Save the model checkpoint every logging step
    save_steps=25,                # Save checkpoints every 25 steps
    eval_strategy="steps", # Evaluate the model every logging step
    eval_steps=25,               # Evaluate and save checkpoints every 25 steps
    do_eval=True,                # Perform evaluation at the end of training
    run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"          # Name of the W&B run 
)

#response_template = " ### Answer:"
#collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

response_template = " [/INST]"
response_template_ids = tokenizer.encode(response_template, add_special_tokens = False)[1:]
collator = DataCollatorForCompletionOnlyLM(response_template=response_template_ids, tokenizer=tokenizer, mlm=False)
trainer = SFTTrainer(
    model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    packing=False, # pack samples together for efficient training
    max_seq_length=tokenizer.model_max_length, # 1024 ?
    args=training_args,
    data_collator=collator
    #data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)



/home/tpllmws23/environments/finetuning/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/tpllmws23/environments/finetuning/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/tpllmws23/environments/finetuning/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:355: UserWarning: You passed a `dataset_kwargs` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/361 [00:00<?, ? examples/s]

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.


Map:   0%|          | 0/91 [00:00<?, ? examples/s]

/home/tpllmws23/environments/finetuning/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:397: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [16]:
response_template = " [/INST]"
response_template_ids = tokenizer.encode(response_template, add_special_tokens = False)[1:]
print(response_template_ids)

[4]


In [17]:
train_dataloader = trainer.get_train_dataloader()

batch = next(iter(train_dataloader))
print(batch)

print(trainer.train_dataset)

trainer.train_dataset[0]['input_ids']

decoded_text = tokenizer.decode(trainer.train_dataset[0]['input_ids'])
print("Decoded Text:", decoded_text)

{'input_ids': tensor([[    1,     3,  1763,  ..., 29491, 29473,     2],
        [    0,     0,     0,  ..., 29491, 29473,     2]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 1, 1, 1]], device='cuda:0'), 'labels': tensor([[ -100,  -100,  -100,  ..., 29491, 29473,     2],
        [ -100,  -100,  -100,  ..., 29491, 29473,     2]], device='cuda:0')}
Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 361
})
Decoded Text: <s>[INST] You are a smart helpful assistant for the HTWG Konstanz. Answer the following question based only on the provided context. Always return the source of an information and it is mandatory to answer in GERMAN:

Context: [['4 ermittelte \nTeilnote 3 abgezogen. Die Zahl der einzuladenden rangbesten Bewerber und Bewerberinnen \nbeträgt das Dr eifache der für die jeweilige Studienrichtung zur Verfügung stehenden \nStudienplätze. (4) Erstellung einer Rangliste für die Auswahlentscheidung nach § 10 \nFü

In [18]:
trainer.train_dataset[0]['input_ids']


decoded_text = tokenizer.decode(batch['input_ids'][0])
print("Decoded Text:", decoded_text)



Decoded Text: <s>[INST] You are a smart helpful assistant for the HTWG Konstanz. Answer the following question based only on the provided context. Always return the source of an information and it is mandatory to answer in GERMAN:

Context: [['1 in einem Studien gang der Fachrichtungen Maschinenbau, \nElektrotechnik oder einer verwandten Fachrichtung. 2. Englischkenntnisse, äquivalent  zu Niveau- Stufe B1 des Europäischen Referenzrahmens \nfür das Lernen und Lehren von Fremdsprachen. Als äquivalent zu einem Zertifikat über die Niveau -Stufe B1 gelten insbesondere folgende Nachweise:  \n- das Schulabschlusszeugnis, aus dem der Besuch des Englischunterrichts bis zum \nErreichen des mittleren Bildungsabschlusses (10. Klasse) bzw. bis zum Erreichen der \nFachhochschulreife hervorgeht oder  \n- ein Notenspiegel, aus dem die bestandene Prüfungsleistung über eine \nLehrveranstaltung im Rahmen des grundständigen St udiums hervorgeht, die die \nenglische Sprache zum Inhalt hatte oder  \n- eine 

In [76]:
batch['input_ids']
batch['labels']
# without datacollator no instructionmasking is done and the llm is trained on the whole input as opposed to

tensor([[    1,     3,  1763,  ..., 29491, 29473,  -100],
        [ -100,  -100,  -100,  ..., 29491, 29473,  -100]], device='cuda:0')

In [19]:
trainer.data_collator

DataCollatorForCompletionOnlyLM(tokenizer=LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-v0.3', vocab_size=32768, model_max_length=2048, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[INST]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[/INST]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	5: AddedToken("[TOOL_CALLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	6

In [21]:
# remove answers
def create_prompt_no_anwer(row):
    row["completion"] = ""
    return {"text": row["prompt"]}

test_dataset = eval_dataset.map(create_prompt_no_anwer)

In [22]:
from finetune_utils import LLMSampleCB
# add wandb table with samples while training to make sure model is not generating gibberish
wandb_callback = LLMSampleCB(trainer, test_dataset, num_samples=10, max_new_tokens=256)
trainer.add_callback(wandb_callback)

In [26]:
# resume specific wandb run
wandb.init(project='teamprojekt-chatbot-pruefungsamt',resume='must', id='az6ub1ph')

wandb: ERROR Control-C detected -- Run data was not synced


In [23]:
# resume training from specific checkpoint
train_result = trainer.train(resume_from_checkpoint=output_dir + '/checkpoint-425') #trainer.train()
#train_result = trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/tpllmws23/environments/finetuning/lib/python3.11/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
25,0.692500,0.432081
50,0.517900,0.374310
75,0.446600,0.360345
100,0.410900,0.336670
125,0.363500,0.325937
150,0.366900,0.317013
175,0.429200,0.317278
200,0.229600,0.325748
225,0.230500,0.316191
250,0.232000,0.316939


/home/tpllmws23/environments/finetuning/lib/python3.11/site-packages/trl/trainer/utils.py:187: UserWarning: Could not find response key `[4]` in the following instance: <s>[INST] You are a smart helpful assistant for the HTWG Konstanz. Answer the following question based only on the provided context. Always return the source of an information and it is mandatory to answer in GERMAN:

Context: [['§ 5 erfüllt . (2) 1Übersteigt die Zahl der geeigneten Bewerbungen die Gesamtzahl der zur Verfügung \nstehenden Studienplätze, so erstellt die Auswahlkommission gemäß § 10  eine Rangliste. § 8 Auswahlkommission  \n(1) 1Zur Vorbereitung der Auswahlentscheidung wird eine Auswahlkommission eingesetzt. (2) 1Der Fakultätsrat der Fakultät, welcher der betreffende Studiengang zugeordnet ist, bestellt \ndie Auswahlkommission. 2Die Auswahlkommission besteht aus mindestens zwei Mitgliedern  \naus der Fakultät . 3Mindestens eines der beiden Mitglieder muss der Gruppe der \nProfessorinnen und Professoren an

  0%|          | 0/10 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad 

...

  0%|          | 0/10 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

OSError: [Errno 28] No space left on device: './finetune_runs/Mistral-7B-v0.3-raft-completion-only-chatbot-pruefungsamt-finetune/checkpoint-425/adapter_model.safetensors' -> '/home/tpllmws23/.local/share/wandb/artifacts/staging/tmpd45c8fzw'

k/internal/sender.py", line 1613, in finish
    self._dir_watcher.finish()
  File "/home/tpllmws23/environments/finetuning/lib/python3.11/site-packages/wandb/filesync/dir_watcher.py", line 402, in finish
    logger.info("scan save: %s %s", file_path, save_name)
Message: 'scan save: %s %s'
Arguments: ('/home/tpllmws23/Chatbot-LLama-Pruefungsamt/Chatbot-Benni/finetune/wandb/run-20240629_154326-az6ub1ph/files/config.yaml', 'config.yaml')
--- Logging error ---
Traceback (most recent call last):
  File "/home/tpllmws23/.pyenv/versions/3.11.7/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/home/tpllmws23/.pyenv/versions/3.11.7/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 28] No space left on device
Call stack:
  File "/home/tpllmws23/.pyenv/versions/3.11.7/lib/python3.11/threading.py", line 1002, in _bootstrap
    self._bootstrap_inner()
  File "/home/tpllmws23/.pyenv/versions/3.11.7/lib/python3.11/threadin

s_74_bd4d11c78f6cf75e0966.table.json')
--- Logging error ---
Traceback (most recent call last):
  File "/home/tpllmws23/.pyenv/versions/3.11.7/lib/python3.11/logging/__init__.py", line 1114, in emit
    self.flush()
  File "/home/tpllmws23/.pyenv/versions/3.11.7/lib/python3.11/logging/__init__.py", line 1094, in flush
    self.stream.flush()
OSError: [Errno 28] No space left on device
Call stack:
  File "/home/tpllmws23/.pyenv/versions/3.11.7/lib/python3.11/threading.py", line 1002, in _bootstrap
    self._bootstrap_inner()
  File "/home/tpllmws23/.pyenv/versions/3.11.7/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/home/tpllmws23/environments/finetuning/lib/python3.11/site-packages/wandb/sdk/internal/internal_util.py", line 48, in run
    self._run()
  File "/home/tpllmws23/environments/finetuning/lib/python3.11/site-packages/wandb/sdk/internal/internal_util.py", line 100, in _run
    self._finish()
  File "/home/tpllmws23/environments/finetuning/

In [24]:
train_result

NameError: name 'train_result' is not defined

In [25]:
metrics = train_result.metrics
max_train_samples = len(train_dataset)
metrics["train_samples"] = min(max_train_samples, len(train_dataset))
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

NameError: name 'train_result' is not defined

# Load Checkpoint and Test

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

#base_model_id = "mistralai/Mistral-7B-v0.3"
#project = "chatbot-pruefungsamt-finetune"

base_model_id = "mistralai/Mistral-7B-v0.3"
project = "chatbot-pruefungsamt-finetune"
run_type = "raft-completion-only"
run_name = base_model_id.split('/')[1] + f"-{run_type}-" + project
output_dir = "./finetune_runs/" + run_name

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True,
)

eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, output_dir + "/checkpoint-350")

In [3]:
def create_eval_prompt(query: str, context: str):
    system_prompt = "You are a smart helpful assistant for the HTWG Konstanz. Answer the following question based only on the provided context. It is mandatory to answer in GERMAN:\n\n"
    return f"[INST]{system_prompt}Context: {context}\n\nQuestion: {query}[/INST]"

In [4]:
data = [{"page_content": "\u00a7 7 Auswahlverfahren\n(1) 1Am Auswahlverfahren nimmt nur teil, wer\na) sich frist - und formgerecht um einen Studienplatz beworben hat,\nb) die Zugangsvoraussetzungen gem\u00e4\u00df \u00a7 59 Absatz 1 LHG i.  V. m. \u00a7 5 erf\u00fcllt .\n(2) 1\u00dcbersteigt die Zahl der geeigneten Bewerbungen die Gesamtzahl der zur Verf\u00fcgung\nstehenden Studienpl\u00e4tze, so erstellt die Auswahlkommission gem\u00e4\u00df \u00a7 10  eine Rangliste.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}, {"page_content": "\n\nNicht zutreffend.\n4. Berufst\u00e4tigkeit und Qualifikationen\nBerufst\u00e4tigkeiten und Weiterbildungen, die nach Abschluss des grundst\u00e4ndigen\nHochschulstudiums gem\u00e4\u00df Abs. 1 Nr. 1 zu Erfahrungen in der Anwendung der\nfachspezifischen Kompetenzen gef\u00fchrt haben, werden bei der Auswahl ber\u00fccksichtigt. Die\ndurch Weiterbildungszertifikate und Arbeitszeugnisse nachgewiesene Gesamtzeit bis maximal\n3 Jahre wird berechnet. Jeder vollst\u00e4ndige Monat wird mit 0,01 bewertet. Ma\u00dfgeblich ist der\nZeitpunkt des Bewerbungsschlusses. Die kumulierte Gesamtzahl bildet die Teilnote 3.\n(3) Kriterien f\u00fcr die Auswahl der Bewerber und Bewerberinnen zu dem Auswahlgespr\u00e4ch nach \u00a7 9a Abs. 1\nNicht zutreffend.\n(4) Erstellung einer Rangliste f\u00fcr die Auswahlentscheidung nach \u00a7 10  F\u00fcr die Auswahlentscheidung wird unter den Bewerbern und Bewerberinnen eine Rangliste\nnach einer Auswahlnote erstellt, die wie folgt ermittelt wird: Von der Teilnote 1 werden die\nTeilnote 2 und die Teilnote 3 abgezogen.\n(5) Ausl\u00e4nderquote gem\u00e4\u00df Anlage 8 HZVO\nNicht zutreffend.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}, {"page_content": "\n\u00a7 9a Auswahlgespr\u00e4ch\n(1) 1Die nach dem Ergebnis der Vorauswahlkriterien (siehe Besonderer Teil \u00a7\u00a7 12- 26)\nrangbesten Bewerberinnen und Bewerber werden zu einem mindestens 15- min\u00fctigen\nAuswahlgespr\u00e4ch eingeladen. 2Die Zahl der einzuladenden Personen betr\u00e4gt mindestens das\nDreifache der zu vergebenden Studienpl\u00e4tze. 3Pro Auswahlg espr\u00e4ch k\u00f6nnen bis zu drei\nBewerberinnen und Bewerber eingeladen werden.\n(2) 1Die Bewerberinnen und Bewerber werden von der Hochschule rechtzeitig \u00fcber die\nModalit\u00e4ten des Auswahlgespr\u00e4chs informiert. 2Der genaue Termin und die Modalit\u00e4ten\nwerden bei elektronischer  Kommunikation sp\u00e4testens sieben Tage und bei postalischer\nKommunikation sp\u00e4testens 14 Tage vor dem Gespr\u00e4ch durch die Hochschule bekannt\ngegeben.\n(3) 1Im Auswahlgespr\u00e4ch wird die Eignung f\u00fcr den gew\u00e4hlten Studiengang und sich\ntypischerweise anschlie\u00dfende Berufst\u00e4tigkeiten bewertet. 2Hierbei k\u00f6nnen unter anderem\nfolgende Kriterien herangezogen werden:\n1. Grad der Motivation f\u00fcr den gew\u00e4hlten Studiengang und sich typischerweise daran\nanschlie\u00dfende Berufst\u00e4tigkeiten,\n2. Kommunikative/s oziale Kompetenzen/technisches Verst\u00e4ndnis ,\n3. Deckungsgrad zwischen den Erwartungen der Bewerberin oder des Bewerbers und\nden tats\u00e4chlichen Inhalten des Studiengangs und sich typischer weise anschlie\u00dfender\nBerufst\u00e4tigkeiten ,\n4. Pers\u00f6nlicher Eindruck , insbesondere  Schl\u00fcssigkeit  der Argumentation,\nSelbstdarstellung und  Ausdrucksweise sowie Stressresistenz.\n3 Die studiengangsspezifischen Kriterien sowie die Bewertung des Auswahlgespr\u00e4chs sind im\nBesonderen Teil (\u00a7\u00a7 12 -26) festgelegt.  4Eine Bewerberi n oder ein Bewerber  hat erfolgreich an  Seite 9 von 43 einem Auswahlgespr\u00e4ch teilgenommen, wenn er/sie die im B esonderen Teil (\u00a7\u00a7  12-26)\nfestgelegte Mindestpunktzahl bzw. Note erreicht hat. 5Erscheint eine Bewerberi n oder ein\nBewerber ohne Nachweis eines wichtigen Grundes nicht zum Gespr\u00e4ch, wird das Gespr\u00e4ch\nals nicht bestanden bewertet; bei Krankheit ist ein \u00e4rztliches Attest vorzulegen. 6Liegt ein\nwichtiger Grund f\u00fcr die Nichtteilnahme vor,  ist die Bewerberin oder der Bewerber zu einem\nsp\u00e4teren Gespr\u00e4chstermin erneut einzuladen.\n(4) 1Zur Durchf\u00fchrung der Auswahlgespr\u00e4che k\u00f6nnen von der Auswahlkommission mehrere\nGespr\u00e4chskommissionen benannt werden. 2Mitglieder der Auswahlkommission k\u00f6nnen\ngleich zeitig Mitglieder einer Gespr\u00e4chskommission sein. 3Jede Gespr\u00e4chskommission besteht\naus zwei geeigneten fachlich qualifizierten Personen, von denen wenigstens eine Professorin\noder ein Professor der Fakult\u00e4t des jeweiligen Studiengangs bzw. der beteiligten Fakult\u00e4ten\ndes Studiengangs sein muss. 4Die wesentlichen Inhalte der Gespr\u00e4che sind zu protokollieren.\n5Aus dem Protokoll muss die Zuordnung der Inhalte jeweils zu einem der in Abs.  3 genannten\nKriterien sowie zu den teilnehmenden Kandidatinnen und Kandidaten ersichtlich sein.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}, {"page_content": "(3) Kriterien f\u00fcr die Auswahl der Bewerber und Bewerberinnen zu dem\nAuswahlgespr\u00e4ch nach \u00a7 9a Abs. 1\nUnter den Bewerbern, die die Zugangsvoraussetzungen gem\u00e4\u00df Abs. 1 erf\u00fcllen, findet zur\nBegrenzung der Teilnehmerzahl an den Auswahlgespr\u00e4chen eine Vorauswahl nach einer\nRangliste statt. Diese Rangliste wird anhand der Teilnote 2 erstellt. Die Zahl der einzuladenden  Seite 36 von 43 rangbesten Bewerber betr\u00e4gt h\u00f6chstens das Dreifache der zur Verf\u00fcgung stehenden\nStudienpl\u00e4tze im Masterstudiengang Unternehmensf\u00fchrung.\n(4) Erstellung einer Rangliste f\u00fcr die Auswahlentscheidung nach \u00a7 10\nF\u00fcr die Auswahlentscheidung wird unter den Bewerbern, die am Auswahlgespr\u00e4ch nach Abs.\n2 Nr. 1 erfolg reich teilgenommen haben, eine Rangliste nach einer Auswahlnote erstellt, in\nwelche die Teilnote 1 und die Teilnote 2 jeweils zu 50 vom Hundert eingehen.\n(5) Ausl\u00e4nderquote gem\u00e4\u00df Anlage 8 HZVO\nNicht zutreffend.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}]
context = [entry['page_content'] for entry in data]
eval_prompt = create_eval_prompt("""Was kommt nach dem Bewerbungsauswahlprozess?""", str(context))
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=300, repetition_penalty=1.15)[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are a smart helpful assistant for the HTWG Konstanz. Answer the following question based only on the provided context. It is mandatory to answer in GERMAN:

Context: ['§ 7 Auswahlverfahren\n(1) 1Am Auswahlverfahren nimmt nur teil, wer\na) sich frist - und formgerecht um einen Studienplatz beworben hat,\nb) die Zugangsvoraussetzungen gemäß § 59 Absatz 1 LHG i.  V. m. § 5 erfüllt .\n(2) 1Übersteigt die Zahl der geeigneten Bewerbungen die Gesamtzahl der zur Verfügung\nstehenden Studienplätze, so erstellt die Auswahlkommission gemäß § 10  eine Rangliste.', '\n\nNicht zutreffend.\n4. Berufstätigkeit und Qualifikationen\nBerufstätigkeiten und Weiterbildungen, die nach Abschluss des grundständigen\nHochschulstudiums gemäß Abs. 1 Nr. 1 zu Erfahrungen in der Anwendung der\nfachspezifischen Kompetenzen geführt haben, werden bei der Auswahl berücksichtigt. Die\ndurch Weiterbildungszertifikate und Arbeitszeugnisse nachgewiesene Gesamtzeit bis maximal\n3 Jahre wird berechnet. Jeder vollständige

In [5]:
data = [{"page_content": "\n\u00a7 4 Sprachkenntnisse\n(1) 1Neben den allgemeinen Zugangsvoraussetzungen (\u00a7 59 LHG) sind f\u00fcr die in \u00a7 1 Abs. 1\nS. 1 genannten Studieng\u00e4nge deutsche Sprach kenntnisse nachzuweisen. 2Diese k\u00f6nnen\ndurch eine deutsche Hochschulzugangsberechtigung (u. a. erfolgreich abgeschlossenes\ngrundst\u00e4ndiges Hochschulstudium) nachgewiesen werden. 3Ferner kann der\nSprachnachweis durch die Vorlage eines der folgenden Dokumente erbracht werden:\n1. Feststellungspr\u00fcfung f\u00fcr ein Bachelorstudium durch Vorlage der Zugangsberechtigung\ndes Studienkollegs an der Hochschule Konstanz,\n2. Test Deutsch als Fremdsprache (TestDaF), sofern im Durchschnitt mindestens die\nStufe TDN 4 erreicht wurde,   Seite 5 von 43 3. Deutsche Sprachpr\u00fcfung f\u00fcr den Hochschulzugang (DSH), sofern die DSH mit\nmindestens der Stufe DSH -2 abgeschlossen wurde,\n4. \u201eTelc Deutsch C1 Hochschule\u201c\noder eine \u00e4quivalente Sprachpr\u00fcfung gem\u00e4\u00df der Rahmenordnung \u00fcber Deutsche\nSprachpr\u00fcfungen f\u00fcr das Studium an deutschen Hochschulen (RO -DT). 4Auf den Nachweis\neiner deutschen Sprachpr\u00fcfung kann bei Bewerber innen und Bewerbern im besonders\nbegr\u00fcndeten Einzelfall verzichtet werden, insbesondere wenn sie die deutsche\nStaatsangeh\u00f6rigkeit besitzen.\n(2) 1Sprachnachweise f\u00fcr den gew\u00e4hl ten Studiengang, die durch die Bewerberin oder den\nBewerber bis zum Bewerbungsschluss nicht vorgelegt werden k\u00f6nnen, k\u00f6nnen bis zum\nVorlesungsbeginn des Semesters gem\u00e4\u00df Terminplan der Hochschule Konstanz, f\u00fcr das  der\nAntrag auf Zulassung gestellt wurde, nachgereicht werden. 2Die Zulassung erfolgt in diesem\nFall gem \u00e4\u00df \u00a7 6  Abs. 5  unter Vorbehalt .\n(3) 1F\u00fcr Zeitstudierende gelten die Regelungen in \u00a7 10 Zulassungs - und\nImmatrikulationsordnung (ZIO) der Hochschule Konstanz.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}, {"page_content": "\n\u00a7 21b Mechatronik (MME) Berufsbegleitendes Studium\n(1) Studiengangspezifische Zugangsvoraussetzungen gem\u00e4\u00df \u00a7 5  Abs. 1\nZugangsvoraussetzungen f\u00fcr den Masterstudiengang Mechatronik sind:\n1. Ein mit der Note 2,9 oder besser abgeschlossenes grundst\u00e4ndiges Hochschulstudium\ngem\u00e4\u00df \u00a7 5 Abs. 1 Nr. 1 in einem Studiengang der Fachrichtungen Systemtechnik,\nMaschinenbau, Elektrotechnik, Fahrzeugtechnik, Mechatronik, Feinwerktechnik oder einer\nverwandten Fachrichtung.\n2. Englischkenntnisse, \u00e4quivalent z u Niveau- Stufe B1 des Europ\u00e4ischen Referenzrahmens\nf\u00fcr das Lernen und Lehren von Fremdsprachen. Als \u00e4quivalent zu einem Zertifikat \u00fcber die\nNiveau -Stufe B1 gelten insbesondere folgende Nachweise:\nI. das Schulabschlusszeugnis, aus dem der Besuch des Englischunterrichts bis zum\nErreichen des mittleren Bildungsabschlusses (10. Klasse) bzw. bis zum Erreichen\nder Fachhochschulreife hervorgeht oder\nII. ein Notenspiegel, aus dem die bestandene Pr\u00fcfungsleistung \u00fcber eine\nLehrveranstaltung im Rahmen des grundst\u00e4ndigen Studiums hervorgeht, die die\nenglische Sprache zum Inhalt hatte oder\nIII. eine Bescheinigung \u00fcber den mindestens sechsmonatigen Aufenthalt an einer Schule, Hochschule oder anderen Bildungsinstitution mit Englisch als\nUnterrichtssprache oder\nIV. eine Bescheinigung \u00fcber den Aufenthalt im englischsprachigen Ausland, der einen Zeitraum von mindestens sechs Monaten bzw. einem Studiensemester umfasst.\nDie Vorlage anderer geeigneter Nachweise ist m\u00f6glich.\n(2) Auswahlkriterien nach \u00a7 9 Abs. 2\n1. Ergebnis eines Auswahlgespr\u00e4chs\nNicht zutreffend.\n2. Leistungen, die mit der Abschlusspr\u00fcfung des grundst\u00e4ndigen Studiums nach Abs. 1\ni. V. m. \u00a7 5 Abs. 1 Nr. 1 nachgewiesen sind\nDie Durchschnittsnote der Abschlusspr\u00fcfung des grundst\u00e4ndigen Hochschulstudiums nach\nAbs. 1 bildet die Teilnote 1 als Basis zur Bestimmung der Auswahlnote.  Abweichend von Satz\n1 bildet in den F\u00e4llen des \u00a7 3 Abs. 2 Nr. 1 Satz 2 die Durchschnittsnote nach \u00a7 3 Abs. 2 Nr. 1 Satz 3 die Teilnote 1. Bei ausl\u00e4ndischen Bildungsnachweisen ist die Durchschnittsnote nach\ndeutsc her Deutung als Teilnote 1 zu ber\u00fccksichtigen.\nZus\u00e4tzlich werden die Einzelnoten folgender F\u00e4cher der Abschlusspr\u00fcfung des grundst\u00e4ndigen Hochschulstudiums, die \u00fcber die Eignung f\u00fcr den gew\u00e4hlten Studiengang\nbesonderen Aufschluss geben, f\u00fcr die Auswahl herangezogen:\n- Technische Mechanik (Dynamik),\n- Elektrotechnik,\n- Messtechnik,\n- Regelungstechnik,\n- Elektrische Antriebe.\nDabei wird eine Note zwischen 1,0 und 1,7 in einem der o. g. F\u00e4cher jeweils mit dem Wert 0,1\nbewertet. Die kumulierte Gesamtzahl bildet die Teil note 2.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}, {"page_content": "\n\u00a7 21a Mechatronik (MME) Vollzeitstudium\n(1) Studiengangspezifische Zugangsvoraussetzungen gem\u00e4\u00df \u00a7 5  Abs. 1\nZugangsvoraussetzungen f\u00fcr den Masterstudiengang Mechatronik sind:\n1. Ein mit der Note 2,9 oder besser abgeschlossenes grundst\u00e4ndiges Hochschulstudium\ngem\u00e4\u00df \u00a7 5 Abs. 1 Nr. 1 in einem Studiengang der Fachrichtungen Maschinenbau,\nElektrotechnik, Fahrzeugtechnik, Mechatronik, Feinwerktechnik oder einer verwandten\nFachrichtung.\n2. Englischkenntnisse, \u00e4quivalent zu Niveau- Stufe B1 des Europ\u00e4ischen Referenzrahmens\nf\u00fcr das Lernen und Lehren von Fremdsprachen. Als \u00e4quivalent zu einem Zertifikat \u00fcber die\nNiveau -Stufe B1 gelten insbesondere folgende Nachweise:\nI. das Schulabschlusszeugnis, aus dem der Besuch des Englischunterrichts bis zum\nErreichen des mittleren Bildungsabschlusses (10. Klass e) bzw. bis zum Erreichen\nder Fachhochschulreife hervorgeht oder\nII. ein Notenspiegel, aus dem die bestandene Pr\u00fcfungsleistung \u00fcber eine\nLehrveranstaltung im Rahmen des grundst\u00e4ndigen Studiums hervorgeht, die die\nenglische Sprache zum Inhalt hatte oder\nIII. eine Bescheinigung \u00fcber den mindestens sechsmonatigen Aufenthalt an einer Schule, Hochschule oder anderen Bildungsinstitution mit Englisch als\nUnterrichtssprache oder\nIV. eine Bescheinigung \u00fcber den Aufenthalt im englischsprachigen Ausland, der einen Zeitraum von mindestens sechs Monaten bzw. einem Studiensemester umfasst.\nDie Vorlage anderer geeigneter Nachweise ist m\u00f6glich.\n(2) Auswahlkriterien nach \u00a7 9 Abs. 2\n1. Ergebnis eines Auswahlgespr\u00e4chs\nNicht zutreffend.\n2. Leistungen, die mit der Abschlusspr\u00fcfung des grundst\u00e4ndigen Studiums nach Abs. 1\ni. V. m. \u00a7 5 Abs. 1 Nr. 1 nachgewiesen sind\nDie Durchschnittsnote der Abschlusspr\u00fcfung des grundst\u00e4ndigen Hochschulstudiums nach\nAbs. 1 bildet die Teilnote 1 als Basis zur Bestimmung der Auswahlnote. Abweichend von Satz\n1 bildet in den F\u00e4llen des \u00a7 3 Abs. 2 Nr. 1 Satz 2 die Durchschnittsnote nach \u00a7 3 Abs. 2 Nr. 1\nSatz 3 die Teilnote 1. Bei ausl\u00e4ndischen Bildungsnachweisen ist die Durchschnittsnote nach\ndeutscher Deutung als Teilnote 1 zu ber\u00fccksichtigen.\nZus\u00e4tzlich werden die Einzelnoten folgender F\u00e4cher der Abschlusspr\u00fcfung des grundst\u00e4ndigen Hochschulstudiums, die \u00fcber die Eignung f\u00fcr den gew\u00e4hlten Studiengang\nbesonderen Aufschluss geben, f\u00fcr die Auswahl herangezogen:\n- Technische Mechanik (Dynamik),\n- Elektrotechnik,\n- Messt echnik,\n- Regelungstechnik,\n- Elektrische Antriebe.\nDabei wird eine Note zwischen 1,0 und 1,7 in einem der o. g. F\u00e4cher jeweils mit dem Wert 0,1\nbewertet. Die kumulierte Gesamtzahl bildet die Teilnote 2.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}, {"page_content": "\n\u00a7 9 Zugangs - und Auswahlkriterien in den Masterstudieng\u00e4ngen\n(1)  1Im Besonderen Teil (\u00a7\u00a7 12 -26) dieser Satzung k\u00f6nnen ein oder mehrere der in Absatz 2\ngenannten Auswahlkriterien als weitere Zugangskriterien festgelegt werden. 2N\u00e4heres\nregelt der B esondere Teil f\u00fcr den jeweiligen Studiengang (\u00a7\u00a7 12 -26).\n(2)  1F\u00fcr die Bildung der Ranglisten f\u00fcr das erste Fachsemester in den Masterstudieng\u00e4ngen\nwird, neben dem Ergebnis des fachlich einschl\u00e4gigen Hochschulabschlusses oder des\ngleichwertigen Abschlusses,  mindestens eines der folgenden  Auswahlkriterien\nber\u00fccksichtigt:\n1. Leistungen, die in dem Studium erbracht wurden, das Voraussetzung f\u00fcr den Zugang\nzu dem Masterstudiengang ist ,   Seite 8 von 43 2. Englischkenntnisse , n\u00e4heres regelt der Besondere Teil f\u00fcr den jeweiligen Studiengang\n(\u00a7\u00a7 12 -26),\n3. Berufst\u00e4tigkeit und Qualifikationen:\na) Art einer abgeschlossenen Berufsausbildung oder einer Berufst\u00e4tigkeit in einem\nanerkannten Ausbildungsberuf  oder eine andere einschl\u00e4gige Berufst\u00e4tigkeit , die \u00fcber\ndie fachspezifische Eignung Auskunft gibt, jeweils  einzeln und in Kombination, und\nb) Qualifikation en, die \u00fcber die fachspezifische Leistung Auskunft geben, jeweils einzeln\noder in Kombination,\n4. das Ergebnis eines fachspezifischen Studieneignungstests ,\n5. das Ergebnis des Auswahlgespr\u00e4chs/anderen m\u00fcndlichen Verfahrens  gem\u00e4\u00df \u00a7 9a ,\n6. ein Motivationsschreiben,\n7. eine schriftliche Abhandlung (Essay).\n2N\u00e4heres sowie die Gewichtung regelt der B esondere Teil f\u00fcr den jeweiligen Studiengang (\u00a7\u00a7\n12-26).\n(2) 1Die Auswahl f\u00fcr h\u00f6here Fachsemester erfolgt gem\u00e4\u00df \u00a7 7 HZG i. V. m. \u00a7 32 HZVO.", "metadata": {"file_path": "/home/tpllmws23/Chatbot-LLama-Pruefungsamt/main_data_filtered/119_ZuSMa_Senat_18012022.pdf"}, "type": "Document"}]
context = [entry['page_content'] for entry in data]
eval_prompt = create_eval_prompt("""Welche Dokumente kÃ¶nnen als Nachweis fÃ¼r deutsche Sprachkenntnisse akzeptiert werden?""", str(context))
model_input = eval_tokenizer(eval_prompt, return_tensors="pt").to("cuda")

ft_model.eval()
with torch.no_grad():
    print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=500, repetition_penalty=1.15)[0], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are a smart helpful assistant for the HTWG Konstanz. Answer the following question based only on the provided context. It is mandatory to answer in GERMAN:

Context: ['\n§ 4 Sprachkenntnisse\n(1) 1Neben den allgemeinen Zugangsvoraussetzungen (§ 59 LHG) sind für die in § 1 Abs. 1\nS. 1 genannten Studiengänge deutsche Sprach kenntnisse nachzuweisen. 2Diese können\ndurch eine deutsche Hochschulzugangsberechtigung (u. a. erfolgreich abgeschlossenes\ngrundständiges Hochschulstudium) nachgewiesen werden. 3Ferner kann der\nSprachnachweis durch die Vorlage eines der folgenden Dokumente erbracht werden:\n1. Feststellungsprüfung für ein Bachelorstudium durch Vorlage der Zugangsberechtigung\ndes Studienkollegs an der Hochschule Konstanz,\n2. Test Deutsch als Fremdsprache (TestDaF), sofern im Durchschnitt mindestens die\nStufe TDN 4 erreicht wurde,   Seite 5 von 43 3. Deutsche Sprachprüfung für den Hochschulzugang (DSH), sofern die DSH mit\nmindestens der Stufe DSH -2 abgeschlossen wurde,\n4. 